In [1]:
# Load contractions model
from pycontractions import Contractions

cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

d:\stage fact checking\sfcenv\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Parameters

In [2]:
file = 'Data/c1.csv'
ngram = (1,3)
mindf = 2
maxdf = 0.8

### Load

In [3]:
import pandas as pd

df = pd.read_csv(file,sep=",")
display(df.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/f...,"Deer baiting, planned for Block Island by DEM,...",Richard August,08/26/2019,"animals,crime,environment,government regulatio...",politifact,TRUE
1,http://data.gesis.org/claimskg/creative_work/8...,Says 'Oregon is one of only three states that ...,Stop Gillnetting Now,05/30/2019,"environment,jobs,message machine 2012,recreation",politifact,FALSE
2,http://data.gesis.org/claimskg/creative_work/f...,NDP: “Canada went from 2.5 million protected l...,New Democratic Party,03/26/2019,"environment,ndp",factscan,OTHER
3,http://data.gesis.org/claimskg/creative_work/0...,Says Texas has so few gas-pump inspectors in t...,Hank Gilbert,03/25/2019,"consumer safety,energy,job accomplishments",politifact,FALSE
4,http://data.gesis.org/claimskg/creative_work/0...,Says Iowa Senate candidate Joni Ernst 'promise...,League of Conservation Voters,03/25/2019,"education,environment",politifact,TRUE


In [30]:
df5 = df.copy().sort_values(by="author")
df6 = df5.loc[(df5['organization'] != 'snopes')]
authors = df6["author"].unique()
#display(df6)

print(df6.shape)
len(authors)

(995, 7)


532

In [5]:
df1 = df.copy()
df1 = df1.loc[df1['rating'] != 'OTHER']
display(df1.head())

,id,text,author,date,keywords,organization,rating
169,http://data.gesis.org/claimskg/creative_work/9...,'RIPTA has really some of the fullest buses fo...,Abel Collins,03/24/2019,"economy,energy,environment,infrastructure,stat...",politifact,MIXTURE
436,http://data.gesis.org/claimskg/creative_work/e...,'We've got to find 2 billion gallons (of water...,Adam Putnam,03/21/2019,"environment,infrastructure",politifact,MIXTURE
675,http://data.gesis.org/claimskg/creative_work/4...,Says he couldn't keep a 42-inch redfish he cau...,Adam Putnam,03/19/2019,oil spill,politifact,TRUE
728,http://data.gesis.org/claimskg/creative_work/a...,The ocean is now 'much more acidic . . . than ...,Al Gore,03/19/2019,"animals,climate change,economy,energy,environm...",politifact,MIXTURE
632,http://data.gesis.org/claimskg/creative_work/1...,Says electric utilities 'are trying to fool yo...,Al Gore,03/19/2019,"corrections and updates,energy,florida amendments",politifact,MIXTURE


In [6]:
t = df1.loc[(df1['rating'] == 'TRUE')]['id'].count()
f = df1.loc[(df1['rating'] == 'FALSE')]['id'].count()
m = df1.loc[(df1['rating'] == 'MIXTURE')]['id'].count()

print('True claims : %d' % t)
print('False claims : %d' % f)
print('Mixture claims : %d' % m)
print('TF claims : %d' % (t+f))

True claims : 146
False claims : 303
Mixture claims : 541
TF claims : 449


### Preprocess

In [7]:
import re

def preprocess(sentence):
    sentence = sentence.lower().strip()
    translator = str.maketrans('’', "'", '')
    sentence = sentence.translate(translator)
    sentence = re.sub("u\.s\.","united states",sentence)
    sentence = list(cont.expand_texts([sentence],precise=True))[0]
    sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
    sentence = re.sub(",",'',sentence)
    sentence = re.sub('\s+', ' ', sentence).strip()
    sentence = re.sub(" s ",' ',sentence)
    if sentence[0:5] == 'says ':
        sentence = sentence[5:]
    return sentence

In [8]:
for index in df1.index:
    df1.at[index,'text'] = preprocess(df1.at[index,'text'])
    rating = df1.at[index,'rating']
    if rating == 'FALSE':
        df1.at[index,'rating'] = 0
    if rating == 'TRUE':
        df1.at[index,'rating'] = 1
    if rating == 'MIXTURE':
        df1.at[index,'rating'] = 2

In [9]:
for index in df1.index:
    if index < 20:
        print("%s\n" % df1.at[index,'text'])

texas wildfires are linked to climate change

more than 27000 rhode island jobs depend on trade with canada

intensifying hurricane willa headed directly toward 10000 migrant caravan path

if oregon legislature simply authorizes 450 million for the crc it will leave oregon holding the bag for a great deal more cost responsibility

under a new federal regulation it will cost pinellas county 31 million to fix a single ditch

i did not i did not i do not say that i do not say that climate change is a hoax perpetrated by the chinese

the chamber of commerce says new carbon regulations will kill 244000 jobs a year and cost average families 1200 a year

texas has so few gas pump inspectors in the field the hebs of this state the brookshires of this state they are calibrating their own gas pumps the texas department of agriculture just sends them the stickers and they calibrate them

her campaign platform includes the biggest investment in new good paying jobs since world war ii

on unemploym

### Create additional dataframes

In [10]:
#df1 = true/false/mixed
#df2 = true/false
#df3 = true-false/mixed

In [11]:
df2 = df1.copy()
df2 = df2.loc[(df2['rating'] == 0) | (df2['rating'] == 1)]
df2.head()

,id,text,author,date,keywords,organization,rating
675,http://data.gesis.org/claimskg/creative_work/4...,he could not keep a 42 inch redfish he caught ...,Adam Putnam,03/19/2019,oil spill,politifact,1
584,http://data.gesis.org/claimskg/creative_work/c...,if we got solar energy from an area of the sou...,Al Gore,03/20/2019,environment,politifact,1
221,http://data.gesis.org/claimskg/creative_work/9...,the proposed gogebic iron ore mine is just up ...,Al Jazeera America,03/23/2019,"energy,environment",politifact,0
698,http://data.gesis.org/claimskg/creative_work/7...,a bill to advance the keystone xl pipeline is ...,Alan Grayson,03/19/2019,"climate change,congressional rules,environment...",politifact,0
519,http://data.gesis.org/claimskg/creative_work/6...,18 percent of our land in our state right now ...,Alberta Darling,03/20/2019,"animals,environment,recreation,state finances",politifact,1


In [12]:
df3 = df1.copy()
for index in df3.index:
    rating = df3.at[index,'rating']
    if rating == 1:
        df3.at[index,'rating'] = 0
    if rating == 2:
        df3.at[index,'rating'] = 1
        
df3.head()

,id,text,author,date,keywords,organization,rating
169,http://data.gesis.org/claimskg/creative_work/9...,ripta has really some of the fullest buses for...,Abel Collins,03/24/2019,"economy,energy,environment,infrastructure,stat...",politifact,1
436,http://data.gesis.org/claimskg/creative_work/e...,we have got to find 2 billion gallons of water...,Adam Putnam,03/21/2019,"environment,infrastructure",politifact,1
675,http://data.gesis.org/claimskg/creative_work/4...,he could not keep a 42 inch redfish he caught ...,Adam Putnam,03/19/2019,oil spill,politifact,0
728,http://data.gesis.org/claimskg/creative_work/a...,the ocean is now much more acidic than it has ...,Al Gore,03/19/2019,"animals,climate change,economy,energy,environm...",politifact,1
632,http://data.gesis.org/claimskg/creative_work/1...,electric utilities are trying to fool you into...,Al Gore,03/19/2019,"corrections and updates,energy,florida amendments",politifact,1


In [13]:
dfs = dict()
dfs['3 Classes'] = df1
dfs['2 Classes'] = df2
dfs['2 Mixed Classes'] = df3

### Vectors

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    #print(vectorizer.get_feature_names())
    return vectors

### Classification

In [15]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

def classify(df, key):
    X = vectorize(df)
    y = df["rating"]
    y = y.astype('int')
    clf = LinearSVC(random_state=0, tol=1e-5)
    k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    score = cross_val_score(clf, X, y, cv=k_fold, scoring='accuracy')
    print('%s - Mean accuracy: %f, Deviation: %f' % (key,score.mean(),score.std()))

In [16]:
for key in dfs:
    classify(dfs[key], key)

3 Classes - Mean accuracy: 0.525297, Deviation: 0.029272
2 Classes - Mean accuracy: 0.670698, Deviation: 0.050161
2 Mixed Classes - Mean accuracy: 0.539385, Deviation: 0.035392


### Feature Selection

In [17]:
from sklearn.feature_selection import mutual_info_classif

def get_top_features(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    feature_scores = mutual_info_classif(vectors, df['rating'].astype('int'))
    
    for score, fname in sorted(zip(feature_scores, vectorizer.get_feature_names()), reverse=True)[:30]:
        print(fname, score)

In [18]:
for key in dfs:
    print("********** %s **********" % key)
    get_top_features(dfs[key])
    print()

********** 3 Classes **********
nation 0.07881630961771457
fight to 0.057073839375179514
as 0.05690748681722502
damaged by hurricane 0.05574960911595728
to pump 0.055664447025921326
five years 0.05525181304604909
per gallon 0.05492239318177128
the worst 0.054783271097938124
obama is 0.054192075677719
nothing 0.05415150854574713
earned 0.05387334467859883
elected 0.05383238984511696
but now 0.052696160103145484
is because of 0.05164624411829588
states has 0.050893829254245126
surface 0.050831632436858065
makes it illegal 0.0500928496677957
ordered the 0.04981301293523899
the highest 0.049607710877638045
stimulus money 0.04912636547507887
fuel efficiency standards 0.04688345125827231
in parts 0.04662546805504575
bottle 0.045965528257335686
repealed 0.0443278200762971
first time 0.044131435277704245
clean water 0.04406187731066602
citizens 0.04376822024963101
hoax invented 0.043694188664122
colorado 0.04355423771224487
dependence on 0.0434618071602586

********** 2 Classes **********
agai